<a href="https://colab.research.google.com/github/Kallol7/PytorchNotes/blob/main/Pretrained/Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoModel, AutoFeatureExtractor, AutoModel, AutoConfig
import os

In [73]:
# https://huggingface.co/zongzhuofan/co-detr-vit-large-coco/resolve/main/pytorch_model.pth

from huggingface_hub import hf_hub_download

REPO_ID = "zongzhuofan/co-detr-vit-large-coco"
FILENAME = "pytorch_model.pth"

model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME, local_dir ="model")

In [86]:
# model_path = "/content/model/pytorch_model.pth"
checkpoint = torch.load(
    model_path, weights_only=False
)

In [98]:
model_path = "/content/model"
os.rename(f"{model_path}/pytorch_model.pth", f"{model_path}/pytorch_model.bin")

In [99]:
model_config = AutoConfig.from_pretrained(REPO_ID)

In [100]:
model_config

DetrConfig {
  "_name_or_path": "zongzhuofan/co-detr-vit-large-coco",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "attention_dropout": 0.0,
  "auxiliary_loss": false,
  "backbone": "resnet50",
  "backbone_config": null,
  "backbone_kwargs": {
    "in_chans": 3,
    "out_indices": [
      1,
      2,
      3,
      4
    ]
  },
  "bbox_cost": 5,
  "bbox_loss_coefficient": 5,
  "class_cost": 1,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "dice_loss_coefficient": 1,
  "dilation": false,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_coefficient": 0.1,
  "giou_cost": 2,
  "giou_loss_coefficient": 2,
  "init_std": 0.02,
  "init_xavier_std": 1.0,
  "is_encoder_decoder": true,
  "mask_loss_coefficient": 1,
  "model_type": "detr",
  "num_channels": 3,
  "num_hidden_layers": 6,
  "num_queries": 100,
  "p

In [101]:
model = AutoModel.from_pretrained(model_path, config = model_config)

Some weights of DetrModel were not initialized from the model checkpoint at /content/model and are newly initialized: ['backbone.conv_encoder.model.bn1.bias', 'backbone.conv_encoder.model.bn1.running_mean', 'backbone.conv_encoder.model.bn1.running_var', 'backbone.conv_encoder.model.bn1.weight', 'backbone.conv_encoder.model.conv1.weight', 'backbone.conv_encoder.model.layer1.0.bn1.bias', 'backbone.conv_encoder.model.layer1.0.bn1.running_mean', 'backbone.conv_encoder.model.layer1.0.bn1.running_var', 'backbone.conv_encoder.model.layer1.0.bn1.weight', 'backbone.conv_encoder.model.layer1.0.bn2.bias', 'backbone.conv_encoder.model.layer1.0.bn2.running_mean', 'backbone.conv_encoder.model.layer1.0.bn2.running_var', 'backbone.conv_encoder.model.layer1.0.bn2.weight', 'backbone.conv_encoder.model.layer1.0.bn3.bias', 'backbone.conv_encoder.model.layer1.0.bn3.running_mean', 'backbone.conv_encoder.model.layer1.0.bn3.running_var', 'backbone.conv_encoder.model.layer1.0.bn3.weight', 'backbone.conv_encode

In [129]:
# height, width don't matter
random_batch = torch.randn(5,3, 190, 190)
model.eval()
model(random_batch).last_hidden_state.shape

torch.Size([5, 100, 256])

In [113]:
len(list(model.children()))

5

In [116]:
lenght_of_child_of_children = [len(list(child.children())) for child in model.children()]
lenght_of_child_of_children

[2, 0, 0, 1, 2]